In [1]:
import pandas as pd
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
import wandb
import pprint
from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
# from utils.raceDB import build_dataset
import utils.rnn_classes as rnn_classes
from utils.raceDB import build_dataset
import importlib
import datetime
from utils.model_saver import model_saver, model_saver_wandb
import utils.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import utils.training_testing_gru_double as training_testing_gru_double

c:\Users\Nick\Documents\GitHub\BetfairSpringCarnival\src


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
torch.set_default_tensor_type(torch.FloatTensor)

In [4]:
torch.__version__

'2.0.1+cu117'

In [5]:
['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']

['dist_last__1',
 'box_last__1',
 'speed_avg_1',
 'split_speed_v1_1',
 'split_speed_avg_1',
 'split_margin_avg_1',
 'margin_avg_1',
 'margin_time_avg_1',
 'RunHomeTime_1',
 'run_home_speed_1',
 'run_home_speed_v1_1',
 'first_out_avg_1',
 'pos_out_avg_1',
 'post_change_avg_1',
 'races_1',
 'wins_1',
 'wins_last_1',
 'weight_',
 'min_time_',
 'min_split_time_',
 'min_split_time_v1',
 'last_start_price',
 'last_start_prob']

In [6]:
def custom_l2(output, target):
    loss = torch.mean(abs(output-target), dim=1)
    return loss

In [7]:
def validation_CLE(x,y):
    loss_t = -torch.log(torch.exp(x)/torch.sum(torch.exp(x), dim=-1, keepdim=True))*y
    return loss_t

In [8]:
tracks = ['Sandown Park',
 'The Meadows',
 'Warragul',
 'Geelong',
 'Sale',
 'Ballarat',
 'Bendigo',
 'Cranbourne']

In [9]:
os.getcwd()
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
date = datetime.datetime.strptime("2022-08-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NSW"]
data_file = 'horsey_racing_v1.fth'
CUDA_LAUNCH_BLOCKING=1
raceDB = build_dataset(data_file, hidden_size , margin_type='boosted_sftmin',v6=True)
raceDB.create_new_weights_v2()


[['RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R', 'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP']]
(443864, 108)
14
Latest date = 2023-08-31 00:00:00
Latest date = 2023-08-31 00:00:00
num_features_per_dog=14


  1%|          | 309/46015 [00:01<02:18, 330.69it/s]

index 6 is out of bounds for dimension 0 with size 5
880012


  2%|▏         | 735/46015 [00:02<01:41, 447.06it/s]

index 8 is out of bounds for dimension 0 with size 6
880560
index 8 is out of bounds for dimension 0 with size 6
880562
index 7 is out of bounds for dimension 0 with size 6
880563
index 9 is out of bounds for dimension 0 with size 6
880564
index 5 is out of bounds for dimension 0 with size 4
880640
index 5 is out of bounds for dimension 0 with size 4
880641


...

index 12 is out of bounds for dimension 0 with size 11
965598
index 9 is out of bounds for dimension 0 with size 8
965615
index 8 is out of bounds for dimension 0 with size 8
965530


100%|█████████▉| 45855/46015 [01:57<00:00, 377.04it/s]

index 10 is out of bounds for dimension 0 with size 6
965776


100%|██████████| 46015/46015 [01:58<00:00, 389.69it/s]
c:\Users\Nick\Documents\GitHub\BetfairSpringCarnival\src\utils\rnn_classes.py:385: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  r.prob = torch.tensor((1/(torch.tensor(r.start_prices)+0.0001))/((1/torch.tensor(r.start_prices)).sum()+0.0001),device=self.device)


number of races = 46015, number of unique dogs = 46891
0                       (Ashburton, 1600)
1                       (Ashburton, 1200)
2                       (Ashburton, 1200)
3                          (Gawler, 2100)
4                       (Ashburton, 2200)
                       ...               
46010                       (Wagga, 1200)
46011    (Riccarton Park Synthetic, 1200)
46012                       (Wagga, 1200)
46013                       (Wagga, 1000)
46014                       (Wagga, 1200)
Length: 46015, dtype: object


In [10]:
races = raceDB.get_race_input(range(1000,1100))

In [11]:
stat_list = ['dist_last__1', 'box_last__1', 'speed_avg_1', 'split_speed_v1_1', 'split_speed_avg_1', 'split_margin_avg_1', 'margin_avg_1', 'margin_time_avg_1', 'RunHomeTime_1', 'run_home_speed_1', 'run_home_speed_v1_1', 'first_out_avg_1', 'pos_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'wins_last_1', 'weight_', 'min_time_', 'min_split_time_', 'min_split_time_v1', 'last_start_price', 'last_start_prob']


In [26]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project='betfair spring carnival', config=config,):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      # stat_mask = [1,1,1,1,1,1]
      # for stat in stat_list:

      #     stat_flag = config[stat]

      #     stat_mask = stat_mask+[stat_flag]

      # print(stat_mask)
      # stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)

      if 'batch_days' in config.keys():
        raceDB.create_test_split_date(config['training_date_end'])
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'])

      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          input_size = raceDB.batches['packed_x'][0].data[0].shape[0]


      print(f"{input_size=}")



      # config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
    

      


      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_BN_double(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'])

      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        # del_keys = ['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0','layer_norm.weight', 'layer_norm.bias']
        # [model_data['model_state_dict'].pop(k) for k in del_keys]
        model.load_state_dict(model_data['model_state_dict'], strict=False)
        config['parent model'] = prev_model_file
        model = model.to(device)
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
        optimizer.load_state_dict(model_data['optim'])
        # optimizer.to(device)
      else:
        optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

      criterion = nn.CrossEntropyLoss(reduction='none')

      model = model.to(device)
      print(model)

      # and use them to train the model
      try:
        # train_double_loss_regular(model, dataset, criterion, optimizer, scheduler, config, crit2=custom_l2)
        training_testing_gru_double.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      except (KeyboardInterrupt) as error:
        print(error)
        print("finished Early")
        
      # dataset.create_hidden_states_dict()
      raceDB.create_hidden_states_dict_v2()
      model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")
      if sweep:
        # raceDB.reset_all_lstm_states
        raceDB.reset_hidden()
    


    # and test its final performance
    #test(model, test_loader)

    return (model,dataset, optimizer)

In [13]:
len(raceDB.raceIDs)
wandb_config_static = {'hidden_size':128,
                        'stats':raceDB.stats_cols,
                        'races':states,
                        'datafile':data_file,
                        'latest_date':raceDB.latest_date,
                        'input_type':'basic',
                        'num_layers':2,
                        'batch_size': 750,
                        'dropout': 0.3,
                        'epochs': 5000,
                        'learning_rate': 0.0001,
                        'optimizer': 'adamW',
                        'f0_layer_size':128,
                        'f1_layer_size':64,
                        'training_date_end':'2023-04-30',
                        'batch_days':365,
            }

In [14]:
wandb_config_static_roi30 = {'hidden_size':128,
                        'stats':raceDB.stats_cols,
                        'races':states,
                        'datafile':data_file,
                        'latest_date':raceDB.latest_date,
                        'input_type':'basic',
                        'num_layers':2,
                        'batch_size': 750,
                        'dropout': 0.3,
                        'epochs': 1000,
                        'learning_rate': 0.001,
                        'optimizer': 'adamW',
                        'f0_layer_size':128,
                        'f1_layer_size':64,
                        'training_date_end':'2021-11-30',
                        'batch_days':365,
                        'dist_last__1': 1,#
                        'box_last__1':1,#
                        'speed_avg_1':1,#
                        'split_speed_v1_1':1,#
                        'split_speed_avg_1':0,
                        'split_margin_avg_1':0,
                        'margin_avg_1':0,
                        'margin_time_avg_1':1,#
                        'RunHomeTime_1':0,
                        'run_home_speed_1':0,
                        'run_home_speed_v1_1':0,
                        'first_out_avg_1':0,
                        'pos_out_avg_1':1,#
                        'post_change_avg_1':1,#
                        'races_1':0,
                        'wins_1':0,
                        'wins_last_1':1,#
                        'weight_':1,#
                        'min_time_':0,
                        'min_split_time_':0, 
                        'min_split_time_v1':1,#
                        'last_start_price':1, #
                        'last_start_prob':0,
            }

In [15]:
wandb_config_static_kindwood = {'hidden_size':128,
                        'stats':raceDB.stats_cols,
                        'races':states,
                        'datafile':data_file,
                        'latest_date':raceDB.latest_date,
                        'input_type':'basic',
                        'num_layers':2,
                        'batch_size': 750,
                        'dropout': 0.3,
                        'epochs': 1200,
                        'learning_rate': 0.0001,
                        'optimizer': 'adamW',
                        'f0_layer_size':128,
                        'f1_layer_size':64,
                        'training_date_end':'2021-11-30',
                        'batch_days':365,
                        'dist_last__1': 1,#
                        'box_last__1':1,#
                        'speed_avg_1':1,#
                        'split_speed_v1_1':1,#
                        'split_speed_avg_1':0,#
                        'split_margin_avg_1':1,
                        'margin_avg_1':1,#
                        'margin_time_avg_1':0,#
                        'RunHomeTime_1':0,
                        'run_home_speed_1':0,
                        'run_home_speed_v1_1':0,
                        'first_out_avg_1':1,#
                        'pos_out_avg_1':0,#
                        'post_change_avg_1':1,#
                        'races_1':1,
                        'wins_1':1,
                        'wins_last_1':0,
                        'weight_':1,#
                        'min_time_':1,#
                        'min_split_time_':0, 
                        'min_split_time_v1':1,#
                        'last_start_price':1, #
                        'last_start_prob':0,
            }

In [16]:
wandb_config_static_kindwood_v2 = {'hidden_size':128,
                        'stats':raceDB.stats_cols,
                        'races':states,
                        'datafile':data_file,
                        'latest_date':raceDB.latest_date,
                        'input_type':'basic',
                        'num_layers':2,
                        'batch_size': 750,
                        'dropout': 0.3,
                        'epochs': 5000,
                        'learning_rate': 0.0002,
                        'optimizer': 'adamW',
                        'f0_layer_size':128,
                        'f1_layer_size':64,
                        'training_date_end':'2022-11-30',
                        'batch_days':365,
                        'dist_last__1': 1,#
                        'box_last__1':1,#
                        'speed_avg_1':1,#
                        'split_speed_v1_1':1,#
                        'split_speed_avg_1':0,#
                        'split_margin_avg_1':1,
                        'margin_avg_1':1,#
                        'margin_time_avg_1':0,#
                        'RunHomeTime_1':0,
                        'run_home_speed_1':0,
                        'run_home_speed_v1_1':0,
                        'first_out_avg_1':1,#
                        'pos_out_avg_1':0,#
                        'post_change_avg_1':1,#
                        'races_1':1,
                        'wins_1':1,
                        'wins_last_1':0,
                        'weight_':1,#
                        'min_time_':1,#
                        'min_split_time_':0, 
                        'min_split_time_v1':1,#
                        'last_start_price':1, #
                        'last_start_prob':0,
            }

In [17]:
race = raceDB.get_race_input([0,1])

In [18]:
y = torch.stack([x.classes for x in race])

In [19]:
model = rnn_classes.GRUNetv3_BN_double(14,128, num_layers=2,fc0_size=128, fc1_size=64).to('cuda:0')

In [20]:
importlib.reload(training_testing_gru)
importlib.reload(training_testing_gru_double)
importlib.reload(rnn_classes)

c:\Users\Nick\Documents\GitHub\BetfairSpringCarnival\src


<module 'utils.rnn_classes' from 'c:\\Users\\Nick\\Documents\\GitHub\\BetfairSpringCarnival\\src\\utils\\rnn_classes.py'>

In [21]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])

In [22]:
torch.cuda.empty_cache()

In [23]:
raceDB.race_prices_to_prob()

In [24]:
count_na = 0
total = 0
for r in raceDB.racesDict.values():
    if r.prob.sum().isnan():
        # print(r.raceid)
        # break
        count_na += 1
    total+=1

print(count_na, total)

0 46015


In [27]:
WANDB_MODE='offline'
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False,prev_model_file='warm-hill-24')

{'hidden_size': 128, 'stats': "[['RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R', 'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP']]", 'races': ['NSW'], 'datafile': 'horsey_racing_v1.fth', 'latest_date': '2023-08-31T00:00:00', 'input_type': 'basic', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 5000, 'learning_rate': 0.0001, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64, 'training_date_end': '2023-04-30', 'batch_days': 365}
5000
{'hidden_size': 128, 'stats': "[['RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R', 'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP']]", 'races': ['NSW'], 'datafile': 'horsey_racing_v1.fth', 'latest_date': '2023-08-31T00:00:00', 'input_type': 'basic', 'num_layers': 2, 'batch_size': 750, 'dropout': 0.3, 'epochs': 5000, 'learning_rate': 0.0001, 'optimizer': 'adamW', 'f0_layer_size': 128, 'f1_layer_size': 64, 'training_date_end': '2023-04-30', 'batch_days': 365}
basic
Train exa

100%|██████████| 46891/46891 [00:12<00:00, 3742.74it/s] 


[(datetime.date(2019, 12, 1), datetime.date(2020, 11, 30)), (datetime.date(2020, 11, 30), datetime.date(2021, 11, 30)), (datetime.date(2021, 11, 30), datetime.date(2022, 11, 30)), (datetime.date(2022, 11, 30), datetime.date(2023, 4, 29))]
2020-11-30
2021-11-30
2022-11-30
2023-04-29
Train examples [5872, 22653, 9908]


100%|██████████| 3/3 [02:53<00:00, 57.86s/it]


Train examples [20236, 35128, 25948]
Train examples [20236, 35128, 25948]
Train examples [5872, 22653, 9908]
Train examples [5872, 22653, 9908]
here
input_size=14
Loading model warm-hill-24, version 450
odict_keys(['gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'gru.weight_ih_l1', 'gru.weight_hh_l1', 'gru.bias_ih_l1', 'gru.bias_hh_l1', 'fc0.weight', 'fc0.bias', 'layer_norm.weight', 'layer_norm.bias', 'layer_norm.running_mean', 'layer_norm.running_var', 'layer_norm.num_batches_tracked', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias', 'price_fc2.weight', 'price_fc2.bias', 'price_fc3.weight', 'price_fc3.bias'])
GRUNetv3_BN_double(
  (gru): GRU(14, 128, num_layers=2, dropout=0.3)
  (relu0): ReLU()
  (fc0): Linear(in_features=2958, out_features=2958, bias=True)
  (layer_norm): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (rl1): ReLU()
  (drop1): Dropout(p=0.3, inplace=False)
  (fc1): Linear(in_fe

...

  0%|          | 20/5000 [14:31<57:00:15, 41.21s/it]

correct = 771.0, len test = 3924
val_idx=range(0, 15561), len_test=15561


  1%|          | 40/5000 [28:40<55:53:13, 40.56s/it]

correct = 788.0, len test = 3924
val_idx=range(0, 15561), len_test=15561


  1%|          | 43/5000 [31:26<65:23:11, 47.49s/it]

In [ ]:
importlib.reload(training_testing_gru_double)

In [ ]:
all_price = training_testing_gru_double.test_model_v3(model,raceDB,nn.CrossEntropyLoss())

In [ ]:
1923/4789

In [ ]:
all_price.correct.sum()

In [ ]:
all_price.one_hot_win.sum()

In [ ]:
all_price.groupby('raceID')['correct'].sum()

In [ ]:
all_price.to_excel('all_price_test.xlsx')

In [ ]:
all_price.raceID.unique().shape

In [ ]:
sweep_config = {"method": "grid"}

metric = {"name": "ROI < 30", "goal": "maximize"}

sweep_config["metric"] = metric


parameters_dict = {
    "optimizer": {"value": "adamW"},
    "batch_days": {"values": [90,180,365,550,10000]},
    "f0_layer_size": {"values": [128]},
    "f1_layer_size": {"values": [64]},
    "dropout": {"values": [0.3]},
    "input_type": {"values": ['basic']},
    "num_layers": {"values": [2]},
    'hidden_size':{'values':[128]},
    "len_data": {"value": len(raceDB.raceIDs)},
    "stats":{"value": raceDB.stats_cols},
    "races":{"value": states},
    "batch_size":{"value": 10},
}

sweep_config["parameters"] = parameters_dict

parameters_dict.update(
    {
        "epochs": {"values": [1000]},
        "validation_split": {"value": 0.1},
        "training_date_end": {"values": 
                                [
                                '2023-04-30',
                                '2023-02-28',
                                '2022-11-30',
                                '2021-11-30'
                                ]},
        "learning_rate": {"values": [0.0003, 0.0001]},
        "label_smoothing": {"values": [0.01,0.0]},
        "loss": {
            "values": [ "CEL"],
        },
    'dist_last__1': {'values': [0, 1]},
    'box_last__1': {'values': [0, 1]},
    'speed_avg_1': {'values': [0, 1]},
    'split_speed_v1_1': {'values': [0, 1]},
    'split_speed_avg_1': {'values': [0, 1]},
    'split_margin_avg_1': {'values': [0, 1]},
    'margin_avg_1': {'values': [0, 1]},
    'margin_time_avg_1': {'values': [0, 1]},
    'RunHomeTime_1': {'values': [0, 1]},
    'run_home_speed_1': {'values': [0, 1]},
    'run_home_speed_v1_1': {'values': [0, 1]},
    'first_out_avg_1': {'values': [0, 1]},
    'pos_out_avg_1': {'values': [0, 1]},
    'post_change_avg_1': {'values': [0, 1]},
    'races_1': {'values': [0, 1]},
    'wins_1': {'values': [0, 1]},
    'wins_last_1': {'values': [0, 1]},
    'weight_': {'values': [0, 1]},
    'min_time_': {'values': [0, 1]},
    'min_split_time_': {'values': [0, 1]},
    'min_split_time_v1': {'values': [0, 1]},
    'last_start_price': {'values': [0, 1]},
    'last_start_prob': {'values': [0, 1]}
}
)

import pprint

pprint.pprint(sweep_config)


sweep_config

In [ ]:
# wandb.agent('x9mz21wl', function=model_pipeline, count=100, project='NEW GRU-sweeps')

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="Variable Sweeps")
# CUDA_LAUNCH_BLOCKING=1
wandb.agent(sweep_id, function=model_pipeline, count=1)